**El COLAB tiene que ser ejecutado en un "ENTORNO T4"**

**Instalo las librerias necesarias para el funcionamiento de LLAMA2**

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.2 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 61.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 8.6 MB/s eta 0:00:00
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-gxfikvv6/overlay/local/bin
    changing mode of /tmp/pip-build-env-gxfikvv6/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-gxfikvv6/overlay/local/bin/wheel to 755
    changing mode of /tmp/pip

**Instalo las librerias necesarias para el funcionamiento de Whisper y Pandas**

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install pandas

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jyif2iya
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jyif2iya
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-m

**Importo las librerias**

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import whisper
import pandas as pd
import sqlite3


**Descargo el modelo LLAMA 2**

In [ ]:
# Nombre del modelo y ruta base del modelo específico que se va a utilizar
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"
# Descarga el modelo desde el repositorio de Hugging Face usando huggingface_hub
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

**Configuración y carga del modelo en GPU LLAMA 2**

In [ ]:
# Inicialización de la variable para el modelo Llama
lcpp_llm = None

# Carga del modelo Llama con parámetros específicos
lcpp_llm = Llama(
    model_path=model_path,  # Ruta del modelo descargado
    n_threads=2,            # Número de núcleos de CPU a utilizar
    n_ctx=4096,             # Tamaño del contexto
    n_batch=512,            # Tamaño del batch, debe estar entre 1 y n_ctx, considerar la cantidad de VRAM en tu GPU
    n_gpu_layers=32         # Número de capas de GPU, ajustar según el modelo y la VRAM disponible en tu GPU
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


**Leer, combinar y limpiar datos de archivos CSV**

In [ ]:
# Leer los archivos CSV en DataFrames
df1 = pd.read_csv("/content/drive/MyDrive/Qpywav/VehicleRecord_Marzo.csv", delimiter=';')
df2 = pd.read_csv("/content/drive/MyDrive/Qpywav/VehicleRecord_Abril.csv", delimiter=';')
df3 = pd.read_csv("/content/drive/MyDrive/Qpywav/VehicleRecord_Mayo.csv", delimiter=';')

# Combinar los DataFrames en uno solo
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Acomodar el DataFrame, limpiarlo y darle formato
combined_df.insert(0, 'ID', range(1, len(combined_df) + 1))  # Insertar columna de ID
combined_df.columns = ['ID', 'PATENTE', 'FECHA', 'TIPO', 'MARCA', 'COLOR']  # Renombrar columnas

# Reemplazar valores en la columna 'TIPO' con valores más descriptivos
replacements = {
    "Salon Car": "Auto", "Motorcycle": "Moto", "Truck": "Camion",
    "Bus": "Colectivo", "Minivan": "Camioneta", "Pedestrian": "Peaton",
    "Tricycle": "Moto"
}
combined_df['TIPO'] = combined_df['TIPO'].replace(replacements)

# Filtrar registros no deseados
combined_df = combined_df[combined_df['TIPO'] != "Other"]

# Convertir la columna 'FECHA' a formato datetime
combined_df['FECHA'] = pd.to_datetime(combined_df['FECHA'], format='%Y/%m/%d %H:%M:%S')

# Mostrar el DataFrame combinado y limpio
print(combined_df)

            ID  PATENTE               FECHA       TIPO       MARCA  \
0            1  Unknown 2024-03-31 23:59:34       Auto       Other   
1            2   BGC049 2024-03-31 23:59:26  Colectivo     Renault   
2            3   JDG077 2024-03-31 23:59:19       Auto       Other   
3            4   IRZ978 2024-03-31 23:57:49       Auto       Other   
4            5  Unknown 2024-03-31 23:57:43       Auto       Other   
...        ...      ...                 ...        ...         ...   
654194  654195   GOS401 2024-05-01 00:01:39       Auto       Other   
654195  654196   FHS617 2024-05-01 00:00:59  Colectivo       Other   
654196  654197  Unknown 2024-05-01 00:00:35       Moto       Other   
654197  654198   JMU125 2024-05-01 00:00:30       Auto  Volkswagen   
654198  654199  AF192SD 2024-05-01 00:00:13       Auto      Toyota   

               COLOR  
0              White  
1              White  
2          Dark Blue  
3             Purple  
4          Dark Blue  
...              ... 

**Creo la Base de datos**

In [ ]:
# Definir el tipo de datos para cada columna en la base de datos SQLite
dtype = {
    'ID': 'INTEGER PRIMARY KEY AUTOINCREMENT',
    'PATENTE': 'TEXT',
    'FECHA': 'DATETIME',
    'TIPO': 'TEXT',
    'MARCA': 'TEXT',
    'COLOR': 'TEXT'
}

# Crear una conexión a la base de datos SQLite
conn = sqlite3.connect('VehiculosDB.db')

# Guardar el DataFrame en una tabla de la base de datos
combined_df.to_sql('vehiculos', conn, if_exists='replace', index=False, dtype=dtype)

654197

Convierto la primer pregunta de audio a texto

In [ ]:
# Definir una función para ejecutar comandos SQL y mostrar resultados

def ejecutar_comando_sql(response, conn):
    # Extraer el comando SQL de la respuesta y limpiar espacios
    sql_command = response["choices"][0]["text"].strip()

    # Buscar la posición de la palabra "SELECT"
    select_index = sql_command.find("SELECT")

    # Si se encuentra "SELECT", cortar la cadena desde esa posición
    if select_index != -1:
        sql_command = sql_command[select_index:].strip()
    else:
        sql_command = sql_command.strip()

    # Ejecutar la consulta SQL y obtener el resultado
    cursor = conn.execute(sql_command)
    QSQL = cursor.fetchall()

    # Imprimir los resultados en pantalla
    print("Resultados:")
    for fila in QSQL:
        print(fila)

    return QSQL

In [ ]:
# Abro el Prompt de contexto para orientar a LLAMA2
# Abre el archivo en modo lectura
with open('/content/drive/MyDrive/Qpywav/SQLPROMPT.txt', 'r', encoding='utf-8') as archivo:
    # Lee todo el contenido del archivo
    prompt_temp = archivo.read()

**Primera consulta**

In [ ]:
# Cargar el modelo de Whisper y transcribir audio

# Cargar el modelo Whisper de tamaño 'medium'
audio_model = whisper.load_model('medium')

100%|█████████████████████████████████████| 1.42G/1.42G [00:18<00:00, 81.8MiB/s]


In [ ]:
# Transcribir el archivo de audio especificado, con idioma en español y utilizando fp16 para acelerar el procesamiento
transcription = audio_model.transcribe("/content/drive/MyDrive/Qpywav/Q1.wav", language='spanish', fp16=True)

# Imprimir la transcripción del texto extraído del audio
print(transcription['text'])

 Cuántos autos motos y camiones pasaron en mayo de 2024


Creo el Prompt para LLAMA 2

In [ ]:
# Crear el prompt para el modelo de lenguaje basado en la transcripción de audio

# Concatenar el texto transcrito con el prefijo 'USER: '
prompt = 'USER: ' + transcription['text']

# Combinar el prompt base con el texto del usuario para formar el prompt completo
prompt_template = prompt_temp + prompt

In [ ]:
# Generar una respuesta utilizando el modelo de lenguaje Llama

# Parámetros del modelo:
# - prompt: El texto de entrada que se utilizará para generar la respuesta.
# - max_tokens: El máximo número de tokens que puede generar el modelo en la respuesta.
# - temperature: Controla la aleatoriedad en la generación del texto. Un valor más alto aumenta la diversidad pero puede generar respuestas menos coherentes.
# - top_p: La probabilidad acumulativa máxima permitida para las predicciones del modelo. Limita las opciones consideradas durante la generación.
# - repeat_penalty: Penalización aplicada a las repeticiones en la generación del texto para evitar la repetición excesiva de palabras o frases.
# - top_k: El número máximo de tokens considerados para la generación del texto, restringiendo las opciones a las más probables.
# - echo: Si se establece en True, incluye el prompt en la salida de la respuesta generada.

response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=512,
    temperature=0.2,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=150,
    echo=False
)

# Imprimir la respuesta generada por el modelo
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit



ASSISTANT: SELECT
  (SELECT COUNT(*) FROM vehiculos WHERE TIPO = 'Auto' AND strftime('%Y-%m', FECHA) = '2024-05') AS Autos,
  (SELECT COUNT(*) FROM vehiculos WHERE TIPO = 'Moto' AND strftime('%Y-%m', FECHA) = '2024-05') AS Motos,
  (SELECT COUNT(*) FROM vehiculos WHERE TIPO = 'Camion' AND strftime('%Y-%m', FECHA) = '2024-05') AS Camiones;


In [ ]:
#Muestro los resultados de la consulta SQL
ejecutar_comando_sql(response, conn)

Resultados:
(144475, 30226, 7495)


[(144475, 30226, 7495)]

**Segunda consulta**

In [ ]:
transcription = audio_model.transcribe("/content/drive/MyDrive/Qpywav/Q2.wav", language='spanish', fp16=True)
print(transcription['text'])

 ¿Cuántos autos con la patente BGC 049 pasaron en abril de 2024?


In [ ]:
prompt = 'USER: ' + transcription['text']
prompt_template = prompt_temp + prompt

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.2, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit



ASSISTANT: SELECT COUNT(*) FROM vehiculos WHERE TIPO = 'Auto' AND PATENTE = 'BGC049' AND strftime('%Y-%m', FECHA) = '2024-04';


In [ ]:
ejecutar_comando_sql(response, conn)

Resultados:
(9,)


[(9,)]

**Tercer consulta**

In [ ]:
transcription = audio_model.transcribe("/content/drive/MyDrive/Qpywav/Q3.wav", language='spanish', fp16=True)
print(transcription['text'])

 Cuántas motos pasaron del 18 de marzo de 2024 al 20 de mayo?


In [ ]:
prompt = 'USER: ' + transcription['text']
prompt_template = prompt_temp + prompt

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.2, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit



ASSISTANT: SELECT COUNT(*) FROM vehiculos WHERE TIPO = 'Moto' AND FECHA BETWEEN '2024-03-18' AND '2024-05-20';


In [ ]:
ejecutar_comando_sql(response, conn)

Resultados:
(76894,)


[(76894,)]

**Cuarta consulta**

In [ ]:
transcription = audio_model.transcribe("/content/drive/MyDrive/Qpywav/Q4.wav", language='spanish', fp16=True)
print(transcription['text'])

 porcentaje de motos pasaron en marzo de 2024


In [ ]:
prompt = 'USER: ' + transcription['text']
prompt_template = prompt_temp + prompt

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.2, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit



ASSISTANT: SELECT (CAST((SELECT COUNT(*) FROM vehiculos WHERE TIPO = 'Moto' AND strftime('%Y-%m', FECHA) = '2024-03') AS FLOAT) / 
                   CAST((SELECT COUNT(*) FROM vehiculos WHERE strftime('%Y-%m', FECHA) = '2024-03') AS FLOAT)) * 100 AS Porcentaje;


In [ ]:
ejecutar_comando_sql(response, conn)

Resultados:
(18.58817395514259,)


[(18.58817395514259,)]